In [1]:
%load_ext autoreload
%autoreload 2

import sys
from datetime import datetime
import time
import random

import ipywidgets as widgets

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

from checkers_load import makeBatches, load_games, dataGenerator, getData
import checkers_models
from checkers_display import *
import checkers_layers

Using TensorFlow backend.


In [2]:
#model = keras.models.load_model(f_name)

figs = []

board_d = SimpleBoardDisplay()
figs.append(board_d.fig)

move_d = MoveHeatmapDisplay()
figs.append(move_d.fig)

pred_d = MoveHeatmapDisplay()
figs.append(pred_d.fig)

display(widgets.HBox(figs))

In [31]:
def getCheckersAdjMat(incSelf=True, incUp=True, incDown=True):
    adj = np.zeros((32,32))

    def cv(x,y):
        return y*4 + x

    def inBnds(x,y):
        return 0 <= x < 4 and 0 <= y < 8

    def conn(x1, y1, x2, y2):
        if inBnds(x1, y1) and inBnds(x2, y2):
            adj[cv(x1,y1), cv(x2,y2)] = 1

    for x in range(4):
        for y in range(8):
            o = (y+1) % 2
            # Connect to ourself
            if incSelf:
                conn(x,y, x,y)
            # Connect down left/right
            if incUp:
                conn(x, y, x+o-1, y+1)
                conn(x+o-1, y+1, x, y)

            if incDown:
                conn(x, y, x+o, y+1)
                conn(x+o, y+1, x, y)


    return adj


# adj = getCheckersAdjMat()
# np.set_printoptions(threshold=np.inf)
# print(adj)
figs = []

b1 = SimpleBoardDisplay()
figs.append(b1.fig)
b2 = SimpleBoardDisplay()
figs.append(b2.fig)
display(widgets.HBox(figs))

In [27]:
# Test adjacency matrix...
adj = getCheckersAdjMat()

a = np.zeros((32,))
i = 24
a[i] = 1
print(a)
print(adj[i])

b1.update(np.reshape(a, (8,4,1)))
b2.update(np.reshape(adj[i], (8,4,1)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0.]


In [4]:
games = load_games()
moves = [a for a in dataGenerator(games)]

In [48]:


# Requires input of shape (32,).
def stupidGraphTestModel():
    # model = Sequential()
    
    adj = getCheckersAdjMat()
    adj = np.identity(32)
    
    
    
    
    k_adj = K.variable(adj)
    fixed_adj = Input(tensor=k_adj)
    print(adj)

    
    X_in = Input(shape=(32,1))

    out = checkers_layers.ConstGraphConv(4, fixed_adj, activation='relu')(X_in)
    out = Flatten()(out)
    
    model = Model(inputs=[X_in], outputs=out)
    optim = Adam(0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=checkers_models.metrics)
    model.summary()

    return model


def literallyFuckingStupidModel(layers):
    model = Sequential()

    model.add(Conv2D(10, (3,3), activation='relu', input_shape=(8,8,1), padding='same'))
    for i in range(layers-1):
        model.add(Conv2D(3, (3,3), activation='relu', padding='same'))

    model.add(Conv2D(4, (1,1), activation='softmax', padding='same'))

    model.add(Flatten(input_shape=(8,8,1)))
    
    optim = Adam(0.0001)
    # model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=metrics)
    model.compile(loss='mse', optimizer=optim, metrics=checkers_models.metrics)
    model.summary()

    return model


m = checkers_models.stupidGraphModel(layers=2)

heyy <tf.Variable 'Variable_39:0' shape=(32, 32) dtype=float32_ref> Tensor("const_graph_conv_66/transpose_2:0", shape=(?, 32, 100), dtype=float32)
(None, 32, 1) 100
heyy <tf.Variable 'Variable_40:0' shape=(32, 32) dtype=float32_ref> Tensor("const_graph_conv_67/transpose_2:0", shape=(?, 32, 100), dtype=float32)
(None, 32, 1) 100
heyy <tf.Variable 'Variable_41:0' shape=(32, 32) dtype=float32_ref> Tensor("const_graph_conv_68/transpose_2:0", shape=(?, 32, 100), dtype=float32)
(None, 32, 1) 100
heyy <tf.Variable 'Variable_39:0' shape=(32, 32) dtype=float32_ref> Tensor("const_graph_conv_69/transpose_2:0", shape=(?, 32, 100), dtype=float32)
(None, 32, 100) 100
heyy <tf.Variable 'Variable_40:0' shape=(32, 32) dtype=float32_ref> Tensor("const_graph_conv_70/transpose_2:0", shape=(?, 32, 100), dtype=float32)
(None, 32, 100) 100
heyy <tf.Variable 'Variable_41:0' shape=(32, 32) dtype=float32_ref> Tensor("const_graph_conv_71/transpose_2:0", shape=(?, 32, 100), dtype=float32)
(None, 32, 100) 100
heyy

In [6]:
X, Y = getData(games)

Size of input is: 1521.05984 MB.


In [7]:
# Data prep

X2 = np.reshape(X, (-1, 32, 1))

print(X2.shape)
print(Y.shape)

(1188328, 32, 1)
(1188328, 128)


In [ ]:
# Training!!
X_train, X_test, Y_train, Y_test = train_test_split(X2, Y, test_size=0.10, random_state=42)

batch_s = 1024
epochs = 10

m.fit(X_train, Y_train,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.1)


Train on 962545 samples, validate on 106950 samples
Epoch 1/10
 88064/962545 [=>............................] - ETA: 8:50 - loss: 4.4622 - acc: 0.0400 - categorical_accuracy: 0.0400 - top_3_accuracy: 0.1816 - top_5_accuracy: 0.2757

In [ ]:
X3 = np.zeros((X.shape[0],8,8,1))
#print(m.shape, m[::2, ::2].shape, m[1::2, 1::2].shape, x0[::2, :].shape, x0[1::2, :].shape)
X3[:, ::2, 1::2] = X[:, ::2, :]
X3[:, 1::2, ::2] = X[:, 1::2, :]

X_train, X_test, Y_train, Y_test = train_test_split(X3, Y, test_size=0.10, random_state=42)

batch_s = 1024
epochs = 10


m = literallyFuckingStupidModel(layers=1)
m.fit(X_train, Y_train,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.1)

In [21]:
# Create UI with 3 boards: Game state, target output, network output.
figs = []

board_d = BoardDisplay(canInteract=False)
figs.append(board_d.fig)

move_d = MoveHeatmapDisplay()
figs.append(move_d.fig)

pred_d = MoveHeatmapDisplay()
figs.append(pred_d.fig)
    
display(widgets.HBox(figs))

5.0
0.1


In [24]:
ix = int(100000*np.random.rand(1))
x_i, y_i = X2[ix], Y[ix]
x_i = np.expand_dims(x_i, axis=0)

#x = np.expand_dims(x, axis=3) # Cause conv2d

y_pred = m.predict(x_i)
y_pred = np.reshape(y_pred, (8,4,4))

board_d.update(X[ix])
move_d.update(y_i)
pred_d.update(y_pred)